In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pyautogui
import time
from selenium.webdriver.support.ui import Select
from getpass import getpass
import pandas as pd

# 1. Entrar no SEI

In [ ]:
driver = webdriver.Chrome()
driver.get("https://sei.al.gov.br/sip/login.php?sigla_orgao_sistema=AL&sigla_sistema=SEI&infra_url=L3NlaS8=")

Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)


# 2. Login

In [ ]:
login = cpf_sei
senha = senha_sei

# 3. Consulta do Processo

# 4. Pegar a unidade atual